In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_rows = 10
pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 600
from tqdm import tqdm
import gc

from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.decomposition import PCA


from tensorflow.keras.layers import BatchNormalization

from keras.models import Sequential, Model

from keras.layers import Input, Embedding, Dense, Activation, Dropout, Flatten

from keras import regularizers

import keras

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GroupKFold

In [ ]:
def init():
    np.random.seed = 0
    
init()

In [ ]:
def smape(y_true,y_pred):
    denominator = (np.abs(y_true)+ np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred)/denominator
    diff[denominator == 0] = 0.0
    return np.nanmean(diff)

def smape2D(y_true,y_pred):
    return smape(np.ravel(y_true),np.ravel(y_pred))

def smape_mask(y_true,y_pred,threshold):
    denominator = (np.abs(y_true) + np.abs(y_pred))
    diff = np.abs(y_true - y_pred)
    diff[denominator == 0] = 0.0
    
    return diff <= (threshold / 2.0) * denominator

In [ ]:
max_size = 181

offset = 1/2
train_all = pd.read_csv('/kaggle/input/web-traffic-time-series-forecasting/train_2.csv.zip')
train_all.head()

In [ ]:
all_page = train_all.Page.copy()
train_key = train_all[['Page']].copy()
train_all = train_all.iloc[:,1:] * offset
train_all.head()

In [ ]:
data = [page.split('_') for page in tqdm(train_key.Page)]

access = ['_'.join(page[-2:]) for page in data]

site = [page[-3] for page in data]

page = ['_'.join(page[:-3]) for page in data]
page[:2]

train_key['PageTitle'] = page
train_key['Site'] = site
train_key['AccessAgent'] = access
train_key.head()